This code snippet is to depict the implementation of the keras method - of faster RCNN



In [ ]:
# os module for file, cv2 is opencv - used for images in python
import os, cv2
# tensorflow for RCNN
import tensorflow as tf
#keras layers is for architecture and models is for forwarding through the architecture
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout
from keras.layers import Flatten, Activation
from keras import Model, Input
#sklearn train-test-split for splitting the data into train and testing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sys

Using TensorFlow backend.


In [ ]:
# pandas for reading csvs
import pandas as pd
import matplotlib.pyplot as plt_curve
import numpy as np
from matplotlib import image

In [ ]:
# since cv2.imshow doesn't work
from google.colab.patches import cv2_imshow

ZF net FrameWork Module

In [ ]:
def zf_module(x,y,x1,y1):
  customArch = Sequential(name="spot the difference")

  # 5 layers in the convolution - for each the dimension changes - upto 256
  customArch.add(Conv2D(16,(3,3), strides=1, padding='same'))
  customArch.add(Conv2D(32,(3,3), strides=1, padding='same'))
  customArch.add(Conv2D(64,(3,3), strides=1, padding='same'))
  customArch.add(Conv2D(128,(3,3), strides=1, padding='same'))
  customArch.add(Conv2D(256,(3,3), strides=1, padding='same'))
  customArch.add(Dropout(0.4))
  customArch.add(MaxPooling2D(2,2))

  customArch.add(Flatten())
  customArch.add(Dense(10, activation = "softmax"))

  customArch.compile(optimizer = "adam",metrics = ["accuracy"])

  customArch.summary()
  return customArch

In [ ]:
# read and preprocess data from the respective csvs
def read_preprocess_data(folder):
  train = pd.read_csv('ObjDetTrain.csv')
  test = pd.read_csv('ObjDetTest.csv')
  train.head()
  print('Uinqueness')
  train['fileName'].nunique()

In [ ]:
# from the whole data split into train and test groups
def getSplitData():
  X = train.iloc[:, :-1].values
  Y = test.iloc[:, -1].values
  x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3,random_state=0)
  return ((x_train, y_train), (x_test, y_test))

In [ ]:
# main driver - invoke the functionalities
global folder
folder = 'sample_data/'
read_preprocess_data(folder)
(train_images,train_labels),(test_images,test_labels) = getSplitData()


In [ ]:
#model is connected to the architecture

model = zf_module(train_images,train_labels, test_images, test_labels)
print(model)
# model can be fit only if it is not None
if model is not None:
  print('hi')
  model.compile(optimizer="adam")
  model.fit(train_images, train_labels, epochs=10)

In [ ]:
# calculate the evalutaion - ROC curve
probability = model.predict_proba(testX)
# consider only the positive probabilities (for ROC- only positive needed)
pred = probability[:,1]
FPRate, TPRate, thresh = roc_curve(testY, pred)
# find auc- area under the curve using the ROC curve result
auc_area = auc(FPRate, TPRate)

#plot the curve
plt.title('RoC - receiver operating characteristic')
plt.plot(FPRate, TPRate, 'b', label = 'AUC = %0.2f' % auc_area)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()